# LLM : Interactive Question-Answering System using Large Language Models

# Step 1: Install necessary env and Import Libaries

In [1]:
# install the datasets
!pip install datasets transformers tensorflow gradio evaluate matplotlib seaborn



In [2]:
!pip install ipywidgets

In [3]:
!pip uninstall -y keras
!pip install tf-keras


Found existing installation: keras 3.11.3
Uninstalling keras-3.11.3:
  Successfully uninstalled keras-3.11.3
  Using cached keras-3.11.3-py3-none-any.whl.metadata (5.9 kB)
Using cached keras-3.11.3-py3-none-any.whl (1.4 MB)


In [4]:
!pip install --upgrade transformers huggingface_hub tf-keras


In [5]:
# Dataset and model handling
from datasets import load_dataset
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

# TensorFlow for training
import tensorflow as tf

# Evaluation metrics
import evaluate

# For interactive demo
import gradio as gr

# Data inspection, analysis and visualization
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns


# Step 2: Load Dataset

In [6]:
# here # Loading first 500 examples of SQuAD v2.0 
dataset = load_dataset("squad_v2", split="train[:500]")
print(dataset[0])


{'id': '56be85543aeaaa14008c9063', 'title': 'Beyoncé', 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".', 'question': 'When did Beyonce start becoming popular?', 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}


# Step3: Tokenize/Preprocess Data

In [7]:
# Load tokenizer
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocessing function
def preprocess(example):
    input_text = f"Question: {example['question']} Context: {example['context']}"
    target_text = example['answers']['text'][0] if example['answers']['text'] else ""
    model_inputs = tokenizer(input_text, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(target_text, max_length=64, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply preprocessing
tokenized_dataset = dataset.map(preprocess)
tokenized_dataset.set_format(type="tensorflow", columns=["input_ids", "attention_mask", "labels"])


# Step 4: Convert to Tensorflow dataset

In [8]:
import numpy as np
import tensorflow as tf

# Convert tokenized columns to NumPy arrays
input_ids = np.array(tokenized_dataset["input_ids"])
attention_mask = np.array(tokenized_dataset["attention_mask"])
labels = np.array(tokenized_dataset["labels"])

# Create TensorFlow dataset
tf_dataset = tf.data.Dataset.from_tensor_slices(
    (
        {"input_ids": input_ids, "attention_mask": attention_mask},
        labels
    )
)

# Shuffle and batch
tf_dataset = tf_dataset.shuffle(100).batch(4)


# Step 5: Load TensorFlow Model


In [9]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


In [10]:
from transformers import TFAutoModelForSeq2SeqLM

model_name = "t5-small"
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name, from_pt=True)  # PyTorch -> TF


outputs = model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95
)



TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


NameError: name 'inputs' is not defined

# Step 6: Fine-Tune the Model

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

# Training step function
@tf.function
def train_step(batch_inputs, batch_labels):
    with tf.GradientTape() as tape:
        outputs = model(
            input_ids=batch_inputs["input_ids"],
            attention_mask=batch_inputs["attention_mask"],
            labels=batch_labels
        )
        loss = outputs.loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop (1 epoch for demo)
for batch in tf_dataset:
    batch_inputs = {"input_ids": batch[0]["input_ids"], "attention_mask": batch[0]["attention_mask"]}
    batch_labels = batch[1]
    loss = train_step(batch_inputs, batch_labels)
    print("Batch loss:", loss.numpy())



# Step 7 : Evaluate Model

In [ ]:
! pip install rouge_score

In [ ]:
pip install transformers==4.35.0


In [ ]:
import evaluate

metric = evaluate.load("rouge")

# Example prediction
input_text = "Question: Where is the University of Notre Dame located? Context: The University of Notre Dame is a private Catholic research university in Notre Dame, Indiana."
inputs = tokenizer(input_text, return_tensors="tf")
outputs = model.generate(**inputs, max_new_tokens=50)
pred = tokenizer.decode(outputs[0], skip_special_tokens=True)

refs = ["Notre Dame, Indiana"]
results = metric.compute(predictions=[pred], references=refs)
print(results)


# Step 8: Gradio Demo

In [11]:
import gradio as gr

def answer_question_tf(question, context):
    input_text = f"Question: {question} Context: {context}"
    inputs = tokenizer(input_text, return_tensors="tf")
    outputs = model.generate(**inputs, max_new_tokens=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

iface = gr.Interface(
    fn=answer_question_tf,
    inputs=["text", "text"],
    outputs="text",
    title="TensorFlow LLM QA Demo"
)

# For Jupyter Notebook inline
iface.launch(inline=True)

# For Colab or public link
# iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
